In [1]:
pip install llama-stack-client==0.2.1 faiss-cpu --quiet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_stack_client import LlamaStackClient
from llama_stack_client.types.shared_params.document import Document as RAGDocument
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger as AgentEventLogger
import os

# Initialize the client
client = LlamaStackClient(base_url="http://localhost:8321")

In [3]:
vector_db_id = "my_documents"

response = client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model="all-MiniLM-L6-v2",
    embedding_dimension=384,
    provider_id="faiss",
)

In [4]:
urls = ["memory_optimizations.rst", "chat.rst", "llama3.rst"]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]

client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)

In [5]:
import os
from llama_stack_client.lib.agents.agent import Agent

rag_agent = Agent(
    client,
    model="Qwen/Qwen3-14B",
    # Define instructions for the agent (system prompt)
    instructions="You are a helpful assistant",
    enable_session_persistence=False,
    # Define tools available to the agent
    tools=[
        {
            "name": "builtin::rag/knowledge_search",
            "args": {
                "vector_db_ids": [vector_db_id],
            },
        }
    ],
)

In [6]:
session_id = rag_agent.create_session("test-session")

In [7]:
user_prompts = [
    "How to optimize memory usage in torchtune? use the knowledge_search tool to get information.",
]

In [8]:
from llama_stack_client.lib.agents.event_logger import EventLogger as AgentEventLogger

for prompt in user_prompts:
    print(f"User> {prompt}")
    response = rag_agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in AgentEventLogger().log(response):
        log.print()

User> How to optimize memory usage in torchtune? use the knowledge_search tool to get information.
inference> <think>
Okay, the user is asking how to optimize memory usage in torchtune. I need to use the knowledge_search tool to find relevant information. Let me start by understanding what torchtune is. Torchtune is a library for training and fine-tuning large-scale models, probably related to PyTorch. Memory optimization in such contexts usually involves techniques like gradient checkpointing, mixed precision training, or optimizing data loading.

The user wants specific steps or best practices for torchtune. Since I don't have prior knowledge on this, I should use the knowledge_search function. The query parameter should be a natural language question. Let me phrase it as "How to optimize memory usage in torchtune?" to get the most relevant results. I'll make sure the function call is correctly formatted in JSON within the tool_call tags.
</think>

<tool_call>
{"name": "knowledge_sea